# Creación de conversaciones ARP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación ARP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [4]:
import random
import re
import pickle
import math
import time

import openai
openai.api_key = "FILL"

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether, ARP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de una comunicación ARP

<center><img src="https://lh3.googleusercontent.com/proxy/5BFPjEuLzirf_rQUpK9oPibFDhfXjP7eez9zJdPXfEl5QZyEp6jLTDSpVc8RBygmwlQ5s4hV-V_muXLdOGmmrsjD-JaqO4HtlQHkO1D8Tj8">
    
<center> Esta imagen es genérica, nosotros crearemos conversaciones que sean tanto una pregunta a broadcast como a una ubicación específica.
<br>
    ---
<br>Una conversación normal ARP tiene una petición (opcode = 1) en la que una fuente pregunta a broadcast o a otro elemento de la red por una IP en concreto, y diciendo la IP a la que comunicar la respuesta.
La respuesta tiene (opcode = 2), y tiene como fuente las respuestas, mientras que como destino tiene al que inicia la conversación.   
<center>---

La primera query (query1) representa una conexión ARP sencilla donde se envía la petición a un equipo específico, la segunda (query2) es más complicada, enviando la solicitud a broadcast.

In [8]:
query1 = "This is a ARP conversation summary:\n"
query1 += 'Source: MAC="c2:3d:19:6c:00:01", IP="10.0.0.1" // Destination: MAC="c2:3c:19:6c:00:01" // Wanted: IP= "10.0.0.2"\n'
query1 += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
query1 += 'pkt_1=scapy.Ether(src="c2:3d:19:6c:00:01", dst="c2:3c:19:6c:00:01")/scapy.ARP(op=1, pdst="10.0.0.2", psrc="10.0.0.1", hwdst="c2:3c:19:6c:00:01", hwsrc="c2:3d:19:6c:00:01")\n'
query1 += 'time.sleep(abs(random.gauss(0, 0.03)))'
query1 += 'pkt_2=scapy.Ether(src="c2:3c:19:6c:00:01", dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="c2:3d:19:6c:00:01",  pdst="10.0.0.1")\n'

In [10]:
query2 = "This is a ARP conversation summary:\n"
query2 += 'Source: MAC="c2:3d:19:6c:00:01", IP="10.0.0.1" // Destination: MAC="FF:FF:FF:FF:FF:FF" // Wanted: IP= "10.0.0.2"\n'
query2 += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
query2 += 'pkt_1=scapy.Ether(src="c2:3d:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="10.0.0.2", psrc="10.0.0.1", hwdst="00:00:00:00:00:00", hwsrc="c2:3d:19:6c:00:01")\n'
query2 += 'time.sleep(abs(random.gauss(0, 0.03)))'
query2 += 'rand_MAC= "02:00:00:%02x:%02x:%02x" % (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))'
query2 += 'pkt_2=scapy.Ether(src=rand_MAC, dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc=rand_MAC, hwdst="c2:3d:19:6c:00:01",  pdst="10.0.0.1")\n'
query2 += 'Take your time (few seconds) to think if the packets and their variables make sense for an ARP converstaion, if not, correct them\n'

## Carga de posibles conversaciones

In [13]:
with open("../../data/Conversations/ARP/Conv_summaries.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()

print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 75


### Carga de posibles respuestas anteriores para encadenarlas

In [16]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [34]:
try:
    responses = pickle.load( open("../../data/Conversations/pickle/ARP_pairs_Aday_validation_v2.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


## Generación de comandos de paquetes de conversación

In [37]:
for i in range(math.ceil(len(Conv_summaries)*0.7), len(Conv_summaries)):
   
    summary = Conv_summaries[i]
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    #summaries = random.choices(Conv_summary,k=summaries_per_request)
    text_sum = ""
    
    if summary.find('FF:FF:FF:FF:FF:FF') == -1: # No esta, es decir, entre disps
        query = query1
        query += "This is a conversation summary:\n"
        query += summary + "\n"
        text_sum += summary + "\n"
        query += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important.\n\n"
        query += "Use the following structure:\n"
        query += "pkt_1=...\n"
        query += "time.sleep(abs(random.gauss(0, 0.03)))\n"
        query += "pkt_2=...\n"
        
        #Query es el mensaje que le envias al modelo
        print(query)

        #Esta es la petición que le mandas al modelo de IA
        completion = openai.completions.create(
            model="gpt-3.5-turbo-instruct", 
            prompt=query,
            max_tokens=2600,
            temperature=0.1)

    else: #si esta, es decir, broadcast
        query=query2
        query += "This is a conversation summary:\n"
        query += summary + "\n"
        text_sum += summary + "\n"
        query += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. This is extremely important.\n\n"
        query += "Use the following structure:\n"
        query += "pkt_1=...\n"
        query += "time.sleep(random.randint(1,10)/1000000.0)\n"
        query += "rand_MAC=...\n"
        query += "pkt_2=...\n"
        #Query es el mensaje que le envias al modelo
        print(query)

        #Esta es la petición que le mandas al modelo de IA
        completion = openai.completions.create(
            model="gpt-3.5-turbo-instruct", 
            prompt=query,
            max_tokens=2600,
            temperature=0.1)


    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/ARP/pickle/ARP_pairs_Aday_validation_v2.pkl", "wb" ) )
    
    exec(responses[-1].completion.choices[0].text)
    # Guardamos los paquetes generados en un pcap
    with open("../../data/Conversations/ARP/pcap/ARP_v3_generated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
        wrpcap(f, pkt_1, append=True)
    
    with open("../../data/Conversations/ARP/pcap/ARP_v3_generated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
        wrpcap(f, pkt_2, append=True)
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 54)
This is a ARP conversation summary:
Source: MAC="c2:3d:19:6c:00:01", IP="10.0.0.1" // Destination: MAC="FF:FF:FF:FF:FF:FF" // Wanted: IP= "10.0.0.2"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.Ether(src="c2:3d:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="10.0.0.2", psrc="10.0.0.1", hwdst="00:00:00:00:00:00", hwsrc="c2:3d:19:6c:00:01")
time.sleep(abs(random.gauss(0, 0.03)))rand_MAC= "02:00:00:%02x:%02x:%02x" % (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))pkt_2=scapy.Ether(src=rand_MAC, dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc=rand_MAC, hwdst="c2:3d:19:6c:00:01",  pdst="10.0.0.1")
Take your time (few seconds) to think if the packets and their variables make sense for an ARP converstaion, if not, correct them
This is a conversation summary:
Source

In [34]:
# Generate random MAC address
import random
mac = [random.randint(0x00, 0xff) for i in range(6)]
mac[0] = (mac[0] & 0xfc) | 0x02 # Set the second bit to 1 to indicate a locally administered MAC address
mac = ":".join(map(lambda x: "%02x" % x, mac))

# Create request packet
request_pkt = scapy.Ether(src=mac, dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="192.168.4.20", psrc="192.168.4.10", hwdst="00:00:00:00:00:00", hwsrc=mac)

# Create reply packet
reply_pkt = scapy.Ether(src=mac, dst=mac)/scapy.ARP(op=2, psrc="192.168.4.20", hwsrc=mac, hwdst=mac, pdst="192.168.4.10")

# Add packets to list
pkt_list = [request_pkt, reply_pkt]

In [60]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: MAC=“f3:a4:b5:c6:d7:e8”, IP=“172.16.3.10” // Destination: MAC=“FF:FF:FF:FF:FF:FF” // Wanted: IP= “172.16.3.20”



import scapy.all as scapy

# Generate random MAC address for the responding end
import random
mac = [random.randint(0x00, 0xff) for i in range(6)]
mac[0] |= 0x02
mac = ":".join(["%02x" % x for x in mac])

# Create request packet
request_pkt = scapy.Ether(src="f3:a4:b5:c6:d7:e8", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="172.16.3.20", psrc="172.16.3.10", hwdst="00:00:00:00:00:00", hwsrc="f3:a4:b5:c6:d7:e8")

# Create reply packet
reply_pkt = scapy.Ether(src=mac, dst="f3:a4:b5:c6:d7:e8")/scapy.ARP(op=2, psrc="172.16.3.20", hwsrc=mac, hwdst="f3:a4:b5:c6:d7:e8", pdst="172.16.3.10")

# Add packets to list
pkt_list = [request_pkt, reply_pkt]


In [62]:
#Esta es la forma que tiene una respuesta del modelo de IA
print(responses[-1].completion)

Completion(id='cmpl-92FB5D7MemVeJpZlaomyjw2L6FG9x', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\nimport scapy.all as scapy\n\n# Generate random MAC address for the responding end\nimport random\nmac = [random.randint(0x00, 0xff) for i in range(6)]\nmac[0] |= 0x02\nmac = ":".join(["%02x" % x for x in mac])\n\n# Create request packet\nrequest_pkt = scapy.Ether(src="f3:a4:b5:c6:d7:e8", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="172.16.3.20", psrc="172.16.3.10", hwdst="00:00:00:00:00:00", hwsrc="f3:a4:b5:c6:d7:e8")\n\n# Create reply packet\nreply_pkt = scapy.Ether(src=mac, dst="f3:a4:b5:c6:d7:e8")/scapy.ARP(op=2, psrc="172.16.3.20", hwsrc=mac, hwdst="f3:a4:b5:c6:d7:e8", pdst="172.16.3.10")\n\n# Add packets to list\npkt_list = [request_pkt, reply_pkt]')], created=1710322027, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=274, prompt_tokens=455, total_tokens=729))


### Generación de paquetes

In [65]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<Ether  dst=FF:FF:FF:FF:FF:FF src=f3:a4:b5:c6:d7:e8 type=ARP |<ARP  op=who-has hwsrc=f3:a4:b5:c6:d7:e8 psrc=172.16.3.10 hwdst=00:00:00:00:00:00 pdst=172.16.3.20 |>>, <Ether  dst=f3:a4:b5:c6:d7:e8 src=e7:44:42:a0:65:a0 type=ARP |<ARP  op=is-at hwsrc=e7:44:42:a0:65:a0 psrc=172.16.3.20 hwdst=f3:a4:b5:c6:d7:e8 pdst=172.16.3.10 |>>]


In [67]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [36]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/ARP/pcap/ARPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)